In [1]:
import os
from PIL import Image
from transformers import AutoModelForCausalLM
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images
import torch
import gc
import traceback

/home/elboardy/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/elboardy/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/elboardy/.local/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
cases_dir = '/media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/'
cache_dir = '/media/RLAB-Disk01/Large-Language-Models-Weights'
model_id = "deepseek-ai/deepseek-vl2-small"
offload_folder = '/media/RLAB-Disk01/Large-Language-Models-Weights'

In [3]:
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
# torch.cuda.set_per_process_memory_fraction(0.9)

In [4]:
# import os

# # Make sure cases_dir is properly defined
# # cases_dir = "your/cases/directory/path"

# deleted_count = 0
# for case in os.listdir(cases_dir):
#     case_dir = os.path.join(cases_dir, case)
#     if os.path.isdir(case_dir):
#         response_path = os.path.join(case_dir, 'deepseek-vl-small.txt')
#         if os.path.exists(response_path):
#             try:
#                 os.remove(response_path)
#                 deleted_count += 1
#                 print(f"Deleted: {response_path}")
#             except Exception as e:
#                 print(f"Error deleting {response_path}: {str(e)}")

# print(f"\nDeleted {deleted_count} response files.")
# print(f"Remaining cases without response file: {len(os.listdir(cases_dir)) - deleted_count}")

In [5]:
cases_to_process = []
for case in os.listdir(cases_dir):
    case_dir = os.path.join(cases_dir, case)
    if os.path.isdir(case_dir):
        response_path = os.path.join(case_dir, 'deepseek-vl-small.txt')
        if not os.path.exists(response_path):
            cases_to_process.append(case)
print(f"Found {len(cases_to_process)} cases to process.")


Found 40 cases to process.


In [6]:
processor = DeepseekVLV2Processor.from_pretrained(model_id)
tokenizer = processor.tokenizer
model: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    use_safetensors=True,
    cache_dir=cache_dir,
    offload_folder=offload_folder,
    offload_state_dict=True, 
    attn_implementation="eager",
)


model = model.eval()


/home/elboardy/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:100002
Add image token = ['<image>'] to the tokenizer
<image>:100003
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:100004
<|/ref|>:100005
<|det|>:100006
<|/det|>:100007
<|grounding|>:100008
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:100009
<|Assistant|>:100010



Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.64s/it]


In [7]:
def build_conversation(case_dir, user_prompt,system_prompt):
    image_files = [f for f in os.listdir(case_dir) if f.lower().endswith('.png')]
    image_paths = [os.path.join(case_dir, f) for f in image_files]
    
    image_files= [image_files[-1]]
    image_paths = [image_paths[-1]]
    # Create image placeholders
    image_tags = " ".join(["<image>"] * len(image_files))
    
    return [

         {
            "role": "<|User|>",
            "content": f"{system_prompt}\n{user_prompt}\n{image_tags}",
            "images": image_paths
        },

        {"role": "<|Assistant|>", "content": ""}
    ]


In [8]:
failed_cases = []


for case in cases_to_process:
    case_dir = os.path.join(cases_dir, case)
    try:
        # Load clinical info
        clinical_path = os.path.join(case_dir, 'diagnostic_prompt.txt')
        if not os.path.exists(clinical_path):
            print(f"Missing clinical info for {case}")
            failed_cases.append(case)
            continue
            
        clinical_info = open(clinical_path).read()
        

        system_prompt = """Consider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.

Use the following structure for the report:

## Radiologist's Report

### Patient Information:
- *Age:* 65
- *Sex:* Male
- *Days from earliest imaging to surgery:* 1
- *Histopathological Subtype:* Glioblastoma
- *WHO Grade:* 4
- *IDH Status:* Mutant
- *Preoperative KPS:* 80
- *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21
- *Preoperative T2/FLAIR Abnormality (cm³):* 36.29
- *Extent of Resection (EOR %):* 100.0
- *EOR Type:* Gross Total Resection (GTR)
- *Adjuvant Therapy:* Radiotherapy (RT) + Temozolomide (TMZ)
- *Progression-Free Survival (PFS) Days:* 649
- *Overall Survival (OS) Days:* 736

#### Tumor Characteristics:

#### Segmentation Analysis:

#### Surgical Considerations:

### Clinical Summary:

### Recommendations:

### Prognostic Considerations:

### Follow-Up Plan:

### Additional Notes*(if any)*:

Ensure all findings from all of the images and clinical data provided. Please mention at the end of the report how many images were reviewed."""

        user_prompt = f"""You will be given batches of images, which are different sequences of MRI scans. 
    The images are for patients who are likely to have a brain tumor. Each image will contain up to 10 slices for 5 different sequences and the segmentation masks for the tumor at the bottom row of the image. 
    Additional clinical data about the patient is: 
    {clinical_info}"""



        # Prepare conversation
        conversation = build_conversation(case_dir, user_prompt,system_prompt)
        pil_images = load_pil_images(conversation)
        
        # Process inputs
        inputs = processor(
            conversations=conversation,
            images=pil_images,
            force_batchify=True,
            system_prompt=system_prompt
        ).to(model.device)

        # Generate embeddings
        inputs_embeds = model.prepare_inputs_embeds(**inputs)

        print(f"Messages prepared for case {case}")
        # Generate response

        generation_config = {
    "max_new_tokens": 4096,  # Maximum number of tokens to generate
    "do_sample": False,  # Use greedy decoding (set to True for sampling)
    "temperature": 0.7,  # Controls randomness (lower = more deterministic)
    "top_p": 0.9,  # Nucleus sampling (cumulative probability threshold)
    "repetition_penalty": 1.2,  # Penalizes repeated tokens
    "early_stopping": True,  # Stop generation if EOS token is generated
    "pad_token_id": tokenizer.eos_token_id,  # Padding token ID
    "bos_token_id": tokenizer.bos_token_id,  # Beginning of sentence token ID
    "eos_token_id": tokenizer.eos_token_id,  # End of sentence token ID
}


        outputs = model.language.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=4096,
            temperature=0.7,
            generation_config=generation_config,
            do_sample=False,
            use_cache=True
        )
        
        # Decode and save response
        response = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)

        print(f"Response generated for case {case}, length: {len(response)} characters")


        with open(os.path.join(case_dir, 'deepseek-vl-small.txt'), 'w', encoding='utf-8') as f:
            f.write(response.split("[/INST]")[-1].strip())
            
        # Cleanup
        del inputs, outputs, response
        gc.collect()
        torch.cuda.empty_cache()

        model._clear_cuda_cache()
        
    except Exception as e:
        error_msg = f"\n\nError processing {case}:\n{traceback.format_exc()}"
        print(error_msg)
        failed_cases.append(case)

# Save failed cases
if failed_cases:
    with open(os.path.join(cases_dir, 'failed_deepseek-vl-small.txt'), 'w') as f:
        f.write("\n".join(failed_cases))

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Messages prepared for case RHUH-0019


Error processing RHUH-0019:
Traceback (most recent call last):
  File "/tmp/ipykernel_155214/3777161059.py", line 95, in <module>
    outputs = model.language.generate(
  File "/home/elboardy/.local/lib/python3.9/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/home/elboardy/.local/lib/python3.9/site-packages/transformers/generation/utils.py", line 1350, in generate
    self._validate_model_kwargs(model_kwargs.copy())
AttributeError: 'NoneType' object has no attribute 'copy'



Error processing RHUH-0001:
Traceback (most recent call last):
  File "/tmp/ipykernel_155214/3777161059.py", line 77, in <module>
    inputs_embeds = model.prepare_inputs_embeds(**inputs)
  File "/home/elboardy/Desktop/repo_folder/Comparative_study_of_LLMs-main/Comparative-study/Deep_seek_vl/deepseek_vl2/models/modeling_deepseek_vl_v2.py", line 378, in prepare_inputs_embeds
    images_feature = self.vision(to